In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats
import math

In [2]:
df = pd.read_csv('ej1.csv')

In [3]:
df.dtypes

Nombre       object
Estatura    float64
Peso        float64
Genero       object
dtype: object

In [4]:
df

,Nombre,Estatura,Peso,Genero
0,Denis,1.72,75.3,M
1,Guadalupe,1.82,81.6,M
2,Alex,1.80,86.1,M
3,Alex,1.70,77.1,M
4,Cris,1.73,78.2,M
5,Juan,1.80,74.8,M
6,Juan,1.80,74.3,M
7,Denis,1.50,50.5,F
8,Alex,1.52,45.3,F
9,Cris,1.62,61.2,F


## Clasificador Bayesiano ingenuo
### La probabilidad conjunta está dada por
$$ P(x_1, ..., x_d, y) = \biggl(\prod_{j=1}^{d} P(x_{j}|y = c)\biggr)P(y = c) $$

## Solución por EMV
### Comenzamos dividiendo nuestro conjunto de datos en las dos clases

In [5]:
df_mujer = df.loc[df['Genero'] == "F"]
df_hombre = df.loc[df['Genero'] == "M"]

In [6]:
df_mujer

,Nombre,Estatura,Peso,Genero
7,Denis,1.50,50.5,F
8,Alex,1.52,45.3,F
9,Cris,1.62,61.2,F
10,Rene,1.67,68.0,F
11,Guadalupe,1.65,58.9,F
12,Guadalupe,1.75,68.0,F


In [7]:
df_hombre

,Nombre,Estatura,Peso,Genero
0,Denis,1.72,75.3,M
1,Guadalupe,1.82,81.6,M
2,Alex,1.80,86.1,M
3,Alex,1.70,77.1,M
4,Cris,1.73,78.2,M
5,Juan,1.80,74.8,M
6,Juan,1.80,74.3,M


### Calculando la probabilidad apriori para ambas clases.
#### Se asumirá una distribución de Bernoulli al ser un problema de clasificación binaria, para la cual el EMV es:
$$ \hat{q}_C = \frac{N_C}{N}$$


In [8]:
q_ber_emv_f = float(len(df_mujer)/len(df))
q_ber_emv_m = float(len(df_hombre)/len(df))
print(f"q para la clase mujer:\n"
      f"{q_ber_emv_f}\n")
print(f"q para la clase hombre:\n"
      f"{q_ber_emv_m}\n")

q para la clase mujer:
0.46153846153846156

q para la clase hombre:
0.5384615384615384



### Estimando la media y varianza para estatura y peso de ambas clases
#### Al ser tanto la estatura como el peso variables continuas se asumirán distribuciones normales, para la cual el EMV para los parámetros.
#### Para la media:
$$\hat{\mu}_{(j|C)} = \frac{1}{n}\sum_{i=1}^{n} x_j^{(i)}$$
#### Para la varianza:
$$\hat{\sigma}_{(j|C)} = \frac{1}{n}\sum_{i=1}^{n} (x_j^{(i)} - \hat{\mu}_{(j|C)})^2$$

In [9]:
def variance(li):
    mu = np.mean(li)   
    return float(sum(list(map(lambda x : (x - mu)**2, li))))/len(li)

In [10]:
dim_atribs = 2
dim_params = 2
dim_clases = 2
gauss_params = np.zeros((dim_atribs,dim_params,dim_clases))
gauss_params[0, 0, 0] = np.mean(df_mujer.Estatura)
gauss_params[0, 1, 0] = variance(df_mujer.Estatura)
gauss_params[1, 0, 0] = np.mean(df_mujer.Peso)
gauss_params[1, 1, 0] = variance(df_mujer.Peso)
gauss_params[0, 0, 1] = np.mean(df_hombre.Estatura)
gauss_params[0, 1, 1] = variance(df_hombre.Estatura)
gauss_params[1, 0, 1] = np.mean(df_hombre.Peso)
gauss_params[1, 1, 1] = variance(df_hombre.Peso)
print("Media de Estatura para mujer: " + str(gauss_params[0, 0, 0]))
print("Media de Estatura para hombre: " + str(gauss_params[0, 0, 1]))
print("Media de peso para mujer: " + str(gauss_params[1, 0, 0]))
print("Media de peso para hombre: " + str(gauss_params[1, 0, 1]))

Media de Estatura para mujer: 1.6183333333333334
Media de Estatura para hombre: 1.7671428571428573
Media de peso para mujer: 58.65
Media de peso para hombre: 78.2


In [11]:
print("Varianza de estatura para mujer: " + str(gauss_params[0, 1, 0]))
print("Varianza de estatura para hombre: " + str(gauss_params[0, 1, 1]))
print("Varianza de peso para mujer: " + str(gauss_params[1, 1, 0]))
print("Varianza de peso para hombre: " + str(gauss_params[1, 1, 1]))

Varianza de estatura para mujer: 0.00744722222222222
Varianza de estatura para hombre: 0.0020204081632653097
Varianza de peso para mujer: 71.00916666666667
Varianza de peso para hombre: 15.765714285714276


### Estimando el valor los parámetros para la distribución categórica del nombre, para cada clase, donde cada nombre corresponde a una categoría.
$$ \hat{q}_k = \frac{1}{n}c_k$$

In [12]:
names = set(df.Nombre)
c_m = dict(zip(names, [0]*len(names)))
c_f = dict(zip(names, [0]*len(names)))
for name in df_mujer.Nombre:
    c_f[name]+=1
for name in df_hombre.Nombre:
    c_m[name]+=1
for k, v in c_f.items():
    c_f[k] = v/len(df_mujer)
for k, v in c_m.items():
    c_m[k] = v/len(df_hombre)
print(c_f)
print(c_m)

{'Alex': 0.16666666666666666, 'Guadalupe': 0.3333333333333333, 'Denis': 0.16666666666666666, 'Cris': 0.16666666666666666, 'Rene': 0.16666666666666666, 'Juan': 0.0}
{'Alex': 0.2857142857142857, 'Guadalupe': 0.14285714285714285, 'Denis': 0.14285714285714285, 'Cris': 0.14285714285714285, 'Rene': 0.0, 'Juan': 0.2857142857142857}


### Definiendo las distribuciones a utilizar
#### Para calcular la probabilidad de que una variable aleatoria X tome un valor entre a y b está dada por:
$$P(a < X < b) = \int_{a}^{b} f(x)dx = F(a) - F(b)$$
#### Donde la función $F(.)$ es la función de distribución acumulativa

In [13]:
#Normal
def prob_normal(mu, sigma, X, a, b):
    return scipy.stats.norm.cdf(X + b, loc=mu, scale=math.sqrt(sigma)) - scipy.stats.norm.cdf(X + a, loc=mu, scale=math.sqrt(sigma))


#Categórica
def prob_categorica(dic, c):
    return dic[c]


### Predicciones usando EMV
$$ \hat{y} = argmax_{c}\biggl(\biggl(\prod_{j=1}^{d} P(x_{j}|y = c)\biggr)P(y = c)\biggr) $$

In [14]:
ejemplos = [['Rene', 1.6800, 65.000], ['Guadalupe', 1.7500, 80.000], ['Denis', 1.8000, 79.000], ['Alex', 1.900, 85.000], ['Cris', 1.6500, 70.000]]
for ejemplo in ejemplos:
    #Mujer
    n1 = q_ber_emv_f*prob_categorica(c_f, ejemplo[0])*prob_normal(gauss_params[0, 0, 0], gauss_params[0, 1, 0], ejemplo[1], -.005, .005)*prob_normal(gauss_params[1, 0, 0], gauss_params[1, 1, 0], ejemplo[2], -.05, .05)
    #Hombre
    n2 = q_ber_emv_m*prob_categorica(c_m, ejemplo[0])*prob_normal(gauss_params[0, 0, 1], gauss_params[0, 1, 1], ejemplo[1], -.005, .005)*prob_normal(gauss_params[1, 0, 1], gauss_params[1, 1, 1], ejemplo[2], -.05, .05)
    print(np.argmax([n1, n2]))
    

0
1
1
1
0


## Solución por MAP
### Calculando la probabilidad apriori para ambas clases
$$ \hat{q}_C = \frac{N_C + \alpha - 1}{N + \beta + \alpha - 2}$$

In [15]:
alfa_ber = 2.
alfa_cat = 2.
q_ber_map_f = float((len(df_mujer) + alfa_ber - 1)/(len(df) + 2))
q_ber_map_m = float((len(df_hombre) + alfa_ber - 1)/(len(df) + 2))

gauss_map_params = np.zeros((dim_atribs,dim_params,dim_clases))
gauss_map_params[0, 0, 0] = 1.5
gauss_map_params[0, 1, 0] = 0.1
gauss_map_params[1, 0, 0] = 70.3
gauss_map_params[1, 1, 0] = 85.0
gauss_map_params[0, 0, 1] = 1.7
gauss_map_params[0, 1, 1] = 0.3
gauss_map_params[1, 0, 1] = 85.5
gauss_map_params[1, 1, 1] = 17.0

### Estimando la media para estatura y peso de ambas clases
$$ \hat{\mu} = \frac{\sigma_{0}^2(\sum_{i = 1}^n x^{(i)}) + \sigma^2\mu_0}{\sigma_{0}^2n + \sigma^2} $$

In [16]:
def normal_map_mean(mu_0, sigma_0, sigma, X):
    return (sigma_0*np.sum(X) + sigma*mu_0)/(sigma_0*len(X) + sigma)

In [17]:
#media para estatura de mujer
gauss_params[0, 0, 0] = normal_map_mean(gauss_map_params[0, 0, 0], gauss_map_params[0, 1, 0], gauss_params[0, 1, 0], df_mujer.Estatura)
#media para peso de mujer
gauss_params[1, 0, 0] = normal_map_mean(gauss_map_params[1, 0, 0], gauss_map_params[1, 1, 0], gauss_params[1, 1, 0], df_mujer.Peso)
#media para estatura de mujer
gauss_params[0, 0, 1] = normal_map_mean(gauss_map_params[0, 0, 1], gauss_map_params[0, 1, 1], gauss_params[0, 1, 1], df_hombre.Estatura)
#media para peso de mujer
gauss_params[1, 0, 1] = normal_map_mean(gauss_map_params[1, 0, 1], gauss_map_params[1, 1, 1], gauss_params[1, 1, 1], df_hombre.Peso)
print(gauss_params)

[[[1.61688258e+00 1.76707832e+00]
  [7.44722222e-03 2.02040816e-03]]

 [[6.00738274e+01 7.90539985e+01]
  [7.10091667e+01 1.57657143e+01]]]


### Estimando el valor los parámetros para la distribución categórica del nombre, para cada clase, donde cada nombre corresponde a una categoría.
$$ \hat{q}_k = \frac{c_k + \alpha_k - 1}{n + \sum_{k = 1}^K \alpha_k - K}$$

In [18]:
c_m_map = dict(zip(names, [0]*len(names)))
c_f_map = dict(zip(names, [0]*len(names)))
for name in df_mujer.Nombre:
    c_f_map[name]+=1
for name in df_hombre.Nombre:
    c_m_map[name]+=1
for k, v in c_f_map.items():
    c_f_map[k] = (v + alfa_cat - 1)/(float(len(df_mujer)) + alfa_cat * float(len(names)) - float(len(names)))
for k, v in c_m_map.items():
    c_m_map[k] = (v + alfa_cat - 1)/(float(len(df_hombre)) + alfa_cat * float(len(names)) - float(len(names)))
print(c_f_map)
print(c_m_map)

{'Alex': 0.16666666666666666, 'Guadalupe': 0.25, 'Denis': 0.16666666666666666, 'Cris': 0.16666666666666666, 'Rene': 0.16666666666666666, 'Juan': 0.08333333333333333}
{'Alex': 0.23076923076923078, 'Guadalupe': 0.15384615384615385, 'Denis': 0.15384615384615385, 'Cris': 0.15384615384615385, 'Rene': 0.07692307692307693, 'Juan': 0.23076923076923078}


### Predicciones usando EMV

In [19]:
for ejemplo in ejemplos:
    #Mujer
    n1 = q_ber_map_f*prob_categorica(c_f_map, ejemplo[0])*prob_normal(gauss_params[0, 0, 0], gauss_params[0, 1, 0], ejemplo[1], -.005, .005)*prob_normal(gauss_params[1, 0, 0], gauss_params[1, 1, 0], ejemplo[2], -.05, .05)
    #Hombre
    n2 = q_ber_map_m*prob_categorica(c_m_map, ejemplo[0])*prob_normal(gauss_params[0, 0, 1], gauss_params[0, 1, 1], ejemplo[1], -.005, .005)*prob_normal(gauss_params[1, 0, 1], gauss_params[1, 1, 1], ejemplo[2], -.05, .05)
    print(np.argmax([n1, n2]))

0
1
1
1
0


### Nota: En las predicciones, 0 representa mujer y 1 representa hombre